# A guide to run a financial simulation using the LLM agent acting as CEO 

In [35]:
### imports 

### config python file to start a simulation on FTX exchange scenario
from simulator.agents import Agent
from simulator.core import Simulation
from simulator.utils import StaticPromptsGenerator
from simulator import utils
import pandas as pd 

from simconfig import base_config
from simulator.logging import logger


In [ ]:
import os
cur_dir = os.getcwd() 
os.chdir(cur_dir.replace("notebooks", ""))
print(os.getcwd())

In [ ]:
%load_ext autoreload
%autoreload 2

### Note that the logs tracks all the simulation activities (prompts, logs, and llm input/output)

The simulation will create three logs: 
```
── simulation_results_llm.json
── simulation_results_logs.json
── simulation_results_prompts.json
```


But we can also setup an dictionary to track only the parameters we change as input (e.g., risk-AVERSION) and output (e.g., trades)

In [38]:
##################### OPTIONAL ############################ 
# The logs are saved in the logs folder with all the information about the simulation, this is just a more simple and clean way to see the results
from collections import defaultdict
results_dict = defaultdict(str)
##################### OPTIONAL ############################


### Let's create the simulation builder first

In [39]:
from simulator.core import SimulationBuilder

## create the simulation here, which we fill with default and ad-hoc parameters.
## Seed : control the randomness of the simulation
simulation_builder = SimulationBuilder(seed=42, verbose=False)


#### Most of the environment, agent settings and prompts are defined by default in the src/config files.

In [ ]:
## Load config for Environment and basic agents
config_data = simulation_builder.sim_config_data
config_data

In [ ]:
## for example all the agents the config are defines as follows 
config_data["agents"].keys()

In [ ]:
## And each agent has some details, about recurrent actions and attributes
config_data["agents"]["trading_branch"]

#### You can replace default values directly in the config_file (src/config/ftx_env.yaml) or at run-time as follows:

In [ ]:
## edit the config_data:
config_data["agents"]["trading_branch"]["attributes"]["money_ask"] = 10000
config_data["agents"]["trading_branch"]

In [ ]:
## edit using the simulation_builder (only implemented for attributes for now):
simulation_builder.update_agent_details("trading_branch", "money_ask", 150_000)
config_data["agents"]["trading_branch"]

In [45]:
##################### OPTIONAL ############################ 
### update the results_dict if you want to keep track of the changes in this notebook
results_dict["trading_branch_money_asked"] = config_data["agents"]["trading_branch"]["attributes"]["money_ask"]
##################### OPTIONAL ############################ 


#### Note on agents: 

- 'CEO' is the only agent impersonated by a LLM 

- 'trading_branch' is a rule-based agents.

- 'events' can be used to model some random events happening in the env 


In [ ]:
## The config contains also simulation parameters 
config_data["simulation_parameters"]

In [ ]:

# The number of simulation steps is given by end_date - start_date:

start_date = pd.to_datetime(config_data["simulation_parameters"]["start_date"])
end_date = pd.to_datetime(config_data["simulation_parameters"]["end_date"])

simulation_days = pd.date_range(
    start=start_date, end=end_date, freq=config_data["simulation_parameters"]["timestep"])

## just one step / one day by default
print(simulation_days)        

#### Each day the agents are executed accordinly the defined env:

In [ ]:
config_data["simulation_parameters"]['agent_order'] 


In [ ]:
## you can modify any of this as well:
config_data["simulation_parameters"]['seed'] = 1
config_data["simulation_parameters"]

In [50]:
##################### OPTIONAL ############################ 
### update the results_dict if you want to keep track of the changes in this notebook
results_dict["seed"] = config_data["simulation_parameters"]['seed']
##################### OPTIONAL ############################ 

### Prompts for User and CEO 

#### Again, the simulation_builder loads the default parameters in the src/simconfig/llm_system_prompt.txt and src/simconfig/agent_prompts.json.



In [ ]:
### Define for each Agent/Event and related action the prompt to use for the LLM, with related placeholders
simulation_builder.users_prompt_data

In [ ]:
### for example when the lender provide an extension for the loan: 
simulation_builder.users_prompt_data["Agents"]["TradingBranch"]["ask_money"]["placeholders"]  
### multiple values in the lists allows for different (random) prompts.


In [ ]:
### we can change prompts as well or add values
simulation_builder.users_prompt_data["Agents"]["TradingBranch"]["ask_money"]["placeholders"].append("Give me some money, please!")
simulation_builder.users_prompt_data["Agents"]["TradingBranch"]["ask_money"]["placeholders"]  


#### The "src/simconfig/llm_system_prompt.txt" is the system prompt for the LLM, where several keywords can be replaced at run-time. Default values are in src/simconfig/base_config.py

In [ ]:
simulation_builder.initial_prompt_ceo
## there are placeholders like {ASSET_NAME}

In [ ]:
### you can modifiy some informations in this template:
from string import Formatter

fieldnames = set([fname for _, fname, _, _ in Formatter().parse(simulation_builder.initial_prompt_ceo) if fname])
fieldnames

In [ ]:
### some defaults are given here:
simulation_builder.pressure_variables

In [ ]:
### you can modify them as follows:
simulation_builder.update_ceo_initialisation_prompt("ASSET_NAME", "MY_COIN")
simulation_builder.pressure_variables

In [ ]:
## or add directly new things to the intial_prompt as well 
simulation_builder.initial_prompt_ceo += "{NEW_PLACEHOLDER}"

simulation_builder.update_ceo_initialisation_prompt("NEW_PLACEHOLDER", "You are the best CEO ever!")
simulation_builder.pressure_variables

In [59]:
##################### OPTIONAL ############################ 
# again you can append some details here
results_dict["ASSET_NAME"] = "MY-COIN"
##################### OPTIONAL ############################ 

## Select the LLM you want to use 

In [ ]:
## we currently support several llm models, you can extend the list you find here:
base_config.MODELS_TYPE

In [61]:
## and set the model you want to use in the CEO agent 
simulation_builder.update_agent_details("CEO", "llm_model", "o1-mini")
results_dict["llm_model"] = "o1-mini" 

In [ ]:
simulation_builder.sim_config_data["agents"]["CEO"]

### Build the simulation

In [ ]:
simulation = simulation_builder.build()

### Run the simulation

In [ ]:
simulation.run()

## Save results

In [65]:
##################### OPTIONAL ############################
## we can track results of some actions for the users like the CEO 
for action in ["evaluate_trader_request"]: 
    outcomes = simulation.extract_informations("CEO", action)
    results_dict[action] = outcomes

##################### OPTIONAL ############################

In [ ]:
## save all data 
output_path = "notebook_logs/"
os.makedirs(output_path, exist_ok=True)
## we save logs and the configs 
simulation.close(output_path + "simulation")

df = pd.DataFrame(results_dict, index=[0])
df.to_csv(output_path + "results.csv", index=False)

In [ ]:
df

## Reset logger at the end of each simulation (I will move the logger inside the simulation object)

In [68]:
from simulator.logging import reset_logger
reset_logger()  